In [2]:
import pandas as pd
%matplotlib inline
import random
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns

In [12]:
articles = pd.read_pickle('ArticleMetadata.pkl')

articles.DatePublished = pd.to_datetime(articles.DatePublished)
articles.Tags = articles.Tags.map(lambda x: str(x))
articles.TagArray = articles.Tags.map(lambda x: x.split(','))
articles.TagArray[0]

['Capitalism',
 'Competition',
 'Property Rights',
 'Entrepreneurship',
 'Free Markets',
 'Market Process',
 'Biographies',
 'Innovation',
 'Arts and Music',
 'Technology']

## TODO: Set Up Scoring Criteria For Articles

In [9]:
from scipy.sparse import hstack

## Classify Articles using KMeans

In [11]:
from __future__ import print_function
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.cluster import KMeans, MiniBatchKMeans
import logging
from optparse import OptionParser
import sys
from time import time
import numpy as np

In [13]:
len(articles)

13835

In [15]:
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english',
                             use_idf=True)
X = vectorizer.fit_transform(articles.FullText)

In [20]:
print("n_samples: %d, n_features: %d" % X.shape)

n_samples: 13835, n_features: 72564


In [24]:
print("Performing dimensionality reduction using LSA")
t0 = time()
    # Vectorizer results are normalized, which makes KMeans behave as
    # spherical k-means for better results. Since LSA/SVD results are
    # not normalized, we have to redo the normalization.
n_components = 1000
svd = TruncatedSVD(n_components)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

X = lsa.fit_transform(X)

print("done in %fs" % (time() - t0))

explained_variance = svd.explained_variance_ratio_.sum()
print("Explained variance of the SVD step: {}%".format(
        int(explained_variance * 100)))

print()

Performing dimensionality reduction using LSA
done in 138.557810s
Explained variance of the SVD step: 49%



In [70]:
labels = articles.Topic
true_k = labels.unique().shape[0]
print("expected clusters: %d" % true_k)

expected clusters: 11


In [78]:
km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1,
                verbose=False)

print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()

print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, km.labels_, sample_size=10000))

print()

Clustering sparse data with KMeans(copy_x=True, init='k-means++', max_iter=100, n_clusters=11, n_init=1,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=False)
done in 2.090s



TypeError: unorderable types: float() > str()

In [79]:
km.labels_

array([ 2, 10,  2, ...,  2,  2,  3], dtype=int32)

In [59]:
# from sklearn.datasets import fetch_20newsgroups
# dataset = fetch_20newsgroups(subset='all', 
#                              shuffle=True, random_state=42)

# print("%d documents" % len(dataset.data))
# print("%d categories" % len(dataset.target_names))
# print()

# labels = dataset.target
# true_k = np.unique(labels).s

18846 documents
20 categories



AttributeError: 'numpy.ndarray' object has no attribute 's'

In [73]:
print(labels.unique())
label_arr = range(0,len(labels))
label_arr

['Economics' 'Policy' 'Politics' 'Entrepreneurship' 'Philosophy' 'Culture'
 'World' 'Justice' nan 'History' 'Technology']


range(0, 13835)

In [74]:
dataset.target

array([10,  3, 17, ...,  3,  1,  7])

### Classify Articles Using Clustering TODO